In [ ]:
#Hoshen Kopelman algorithm with periodic boundary conditions
#counting islands of spins up or down and their size. 
#importing necessary libraries
import numpy as np
import random
import matplotlib.pyplot as plt
import numba
from numba import njit
from scipy.ndimage import convolve, generate_binary_structure
plt.style.use(['science','notebook', 'grid'])

In [ ]:
def lattice(N): #create lattice with half spin up and half spin down randomly distributed 
    lattice = np.zeros((N, N))
    random = np.random.random((N, N))
    lattice[random > 0.5 ] = 1
    lattice[random <= 0.5 ] = -1
    return lattice

In [ ]:
#termalize lattice 
@numba.njit(nopython=True)
def MCS_lattice_many(lattice, N, BJ, st): #st is a number of MCS/spin
    lattice = lattice.copy()
    for a in range(0, st-1):
        for b in range(N**2):
            i = random.randint(0,N-1)
            j = random.randint(0,N-1)
            initial = lattice[i][j]
            neigbours = lattice[i][(j-1)%N] + lattice[(i-1)%N][j] + lattice[i][(j+1)%N] + lattice[(i+1)%N][j]
            delta_E =  2 *  initial * neigbours
            if delta_E <= 0:
                lattice[i][j] = (-1) * initial 
            elif (delta_E > 0) * (random.random() < np.exp(- BJ * delta_E )): 
                lattice[i][j] = (-1) * initial     
    return(lattice)

In [ ]:
#Hoshen Kopelman algorithm with periodic boundary conditions
#with arguments: lattice, size of a lattice N
#returns map of indexed islands, island sizes with their frequencies 
def Hoshen_Kopelman_algorithm(arr, N):
    arr = arr.copy()
    largest_label= 1
    unije = np.arange(N*N/2)
    unije_m = np.arange(N*N/2)
    label_table = np.zeros((N, N))
    label_table_m = np.zeros((N, N))
    for i in range(N): #index moves over lattice 
        for j in range(N):
            above = 0 #initially set neighbours
            left = 0
            right = 0
            down = 0

            if arr[i][j] == 1.: #if spin is up, check if the neighbour on the left and on top are the same
                neighbour_table = []
                if i != 0: 
                    above = label_table[i-1][j]
                if i == (N-1): #because of periodic boundary conditions
                    down = label_table[0][j]
                if j != 0: 
                    left = label_table[i][j-1]
                if j == (N-1): #because of periodic boundary conditions
                    right = label_table[i][0]
                np.array(neighbour_table.extend([above, down, left, right]))
                neighbour_table = np.array(neighbour_table)

                if left == 0 and above == 0 and right == 0 and down == 0:
                    label_table[i][j] = largest_label #if spin is sourrounded by opposite spins
                    largest_label += 1                #lable it as new island

                elif left != 0 or above != 0 or right != 0 or down != 0:
                    label_table[i][j] = int(np.min(neighbour_table[neighbour_table > 0]))
                    for k in neighbour_table[neighbour_table > 0]:
                        if k != int(np.min(neighbour_table[neighbour_table > 0])):
                            unije[int(k)] = int(np.min(neighbour_table[neighbour_table > 0]))

            elif arr[i][j] == -1.: #do the same with down spins
                neighbour_table_m = []
                if i != 0:
                    above = label_table_m[i-1][j]
                if i == (N-1): #because of periodic boundary conditions
                    down = label_table_m[0][j]
                if j != 0:
                    left = label_table_m[i][j-1]
                if j == (N-1): #because of periodic boundary conditions
                    right = label_table_m[i][0]
                np.array(neighbour_table_m.extend([above, down, left, right]))
                neighbour_table_m = np.array(neighbour_table_m)

                if left == 0 and above == 0 and right == 0 and down == 0:
                    label_table_m[i][j] = largest_label
                    largest_label += 1
                elif left != 0 or above != 0 or right != 0 or down != 0:
                    label_table_m[i][j] = int(np.min(neighbour_table_m[neighbour_table_m > 0]))
                    for k in neighbour_table_m[neighbour_table_m > 0]:
                        if k != int(np.min(neighbour_table_m[neighbour_table_m > 0])):
                            unije_m[int(k)] = int(np.min(neighbour_table_m[neighbour_table_m > 0]))
                

    for i in range(N):
        for j in range(N):
            while unije[int(label_table[i][j])] != label_table[i][j]:
                label_table[i][j] = unije[int(label_table[i][j])]
            while unije_m[int(label_table_m[i][j])] != label_table_m[i][j]:
                label_table_m[i][j] = unije_m[int(label_table_m[i][j])]
    label_table_final = label_table + label_table_m
    (uniq_k, freq_k) = (np.unique(label_table_final, return_counts=True))
    (uniq2, freq2) = (np.unique(freq_k, return_counts=True))

    return label_table_final, uniq2, freq2


In [ ]:
#compute island sizes and their frequencies for i square lattices of site N
def get_K_H_points(N, i, st, BJ):
    M = int(N*N)
    x = np.arange(M) / M
    y = np.zeros(M)
    for a in range(i):
        current = MCS_lattice_many(lattice(N), N, BJ, st)
        a1, a2, a3 = Hoshen_Kopelman_algorithm(current, N)
        for count, value in enumerate(a2):
            y[int(value)] = a3[int(count)]
    y = y/ y.sum()
    return x, y

#calculate points and save them
x, y = get_K_H_points(100, 300, 90000, 0.4397)
np.savetxt('Hoshen_Kopelman_algorithm_popravljen_oktober', np.column_stack([x, y]), header='P(S), S, lattice 100x100, 300 realisations after termalisation of 90000 steps at BJ=0.439')

In [ ]:
#combining 200 values to create 50 bins
#insert before calculated values and bin them
def bined(z, t):
    f = []
    for a in range(50):
        vsota = 0
        for b in range(200):
            mesto = 200 * a + b
            vsota += t[mesto]
        f.append(vsota)
    return f


d = bined(x, y) #plot histogram
e = np.arange(0, 1, 0.02)
plt.plot(np.log(e[:35]), np.log(d[:35]), 'o', label="calculated values")

z = (np.log(e[2:19])) #linear fit of log values
t = (np.log(d[2:19]))
a, b = np.polyfit(z, t, 1) 
plt.plot(z, a*z+b, label="linear fit")

plt.xlabel(r'$log(S)$')
plt.ylabel(r'$log(P(S))$')
plt.legend()
plt.savefig('kopelman_algorithm_300lattices_size100')

#slope  -1.9438103578170876
